# Monad

Functor => Applicative => Monad

In [3]:
(*) <$> Just 2 <*> Just 8
(++) <$> Just "klingon" <*> Nothing
(-) <$> [3,4] <*> [1,2,3]  

Just 16

Nothing

[2,1,0,3,2,1]

In [ ]:
(>>=) :: (Monad m) => m a -> (a -> m b) -> m b
-- m a (m이라는 컨텍스트로 싸인 a 값에)
-- (a -> m b) (함수를 적용하고 싶다면)

-- m 컨텍스트는 모나드여야한다

1) 컨텍스트로 꾸며진 값과 
2) 일반적인 값을 전달받아서 반환하는 함수가 있을 때

**그 함수에 꾸며진 값을 어떻게 전달할까? (어떻게 컨텍스트를 벗겨서 전달할까?)**

다만, 모나드는 단지 >>= 를 지원하는 Applicative Functor 일 뿐이다.

`>>=` 함수는 bind라고 불린다

## With Maybe

Maybe는 모나드이다

In [5]:
fmap (++"!") (Just "wisdom")  
fmap (++"!") Nothing  

-- Nothing에는 함수가 매핑되지 않는다. 왜냐하면 매핑될 것이 없기 때문이다

Just "wisdom!"

Nothing

In [7]:
Just (+3) <*> Just 3  
Nothing <*> Just "greed"  
Just ord <*> Nothing

Just 6

Nothing

: 

In [8]:
-- Maybe는 또한 어플리커티브 펑터로써, 함수를 적용할수 없을 때 실패함을 알려야한다.
max <$> Just 3 <*> Just 6  
max <$> Just 3 <*> Nothing  

Just 6

Nothing

In [15]:
(\x -> Just (x+1)) 1
(\x -> Just (x+1)) 100
-- (a -> m b) 타입 함수

Just 2

Just 101

In [16]:
applyMaybe :: Maybe a -> (a -> Maybe b) -> Maybe b
-- 타입자체는 모나드의 bind와 같다
applyMaybe Nothing f = Nothing
applyMaybe (Just x) f = f x

In [21]:
Just 3 `applyMaybe` \x -> Just (x+1)
Just "smile" `applyMaybe` \x -> Just (x ++ " :)")
Nothing `applyMaybe` \x -> Just (x+1)
Nothing `applyMaybe` \x -> Just (x++ " :)")

Just 4

Just "smile :)"

Nothing

Nothing

In [22]:
Just 3 `applyMaybe` \x -> if x > 2 then Just x else Nothing
Just 1 `applyMaybe` \x -> if x > 2 then Just x else Nothing

Just 3

Nothing

## 모나드 타입 클래스

In [ ]:
class Monad m where  
-- 타입 클래스 제약이 없지만, 모두가 모든 모나드는 어플리커티브 펑터임을 지키고 있다
    return :: a -> m a  
    -- Applicative Functor의 pure와 같다 (컨텍스트로 래핑하는 함수)
    -- IO 컨텍스트를 다루기 위해 사용했던 그 return 함수 맞다
    (>>=) :: m a -> (a -> m b) -> m b  
    -- 모나드 값과 일반값을 받아 모노이드 값을 반환하는 함수를 받아 모노이드 값을 반환
    (>>) :: m a -> m b -> m b  
    x >> y = x >>= \_ -> y  
    -- 디폴트 구현체가 있다
    -- 인스턴스에서 구현할 필요가 거의 없는 함수
  
    fail :: String -> m a  
    fail msg = error msg  
    -- 반환형은 모나드 값
    -- 코드에서는 명시적으로 사용하지 않음
    -- 모나드를 위한 특별 구문 구조의 실패를 가능하기 위해 하스켈에 의해 사용된다

In [ ]:
instance Monad Maybe where  
    return x = Just x  
    -- 컨텍스트로 감싸기
    Nothing >>= f = Nothing  
    Just x >>= f  = f x 
    -- 바인드 연산
    fail _ = Nothing  
    -- fail 시에는 Nothing 컨텍스트 반환

In [28]:
return "WHAT" :: Maybe String
Just 9 >>= \x -> return (x * 10)
Nothing >>= \x -> return (x * 10)

Just "WHAT"

Just 90

Nothing

## 반복적인 >>= 사용

In [13]:
type Birds = Int
type Pole = (Birds, Birds)

In [14]:
landLeft :: Birds -> Pole -> Pole
landLeft n (left, right) = (left + n, right)

landRight :: Birds -> Pole -> Pole
landRight n (left, right) = (left, right + n)

In [32]:
landLeft 2 (0, 0)
landRight 1 (1, 2)
landRight (-1) (1, 2)

(2,0)

(1,3)

(1,1)

In [33]:
landLeft 2 (landRight 1 (landLeft 1 (0,0)))

(3,1)

In [8]:
x |> f = f x

In [37]:
100 |> (*3)

300

In [39]:
True |> not

False

In [40]:
(0, 0) |> landLeft 2

(2,0)

In [42]:
(0, 0) |> landLeft 1 |> landRight 1 |> landLeft 2

(3,1)

In [44]:
(0, 3) |> landLeft 10 

(10,3)

In [45]:
(0, 0) |> landLeft 1 |> landRight 4 |> landLeft (-1) |> landRight (-2)
-- 3번째 함수에서 에러가 발생해야함

(0,2)

In [16]:
landLeft :: Birds -> Pole -> Maybe Pole
landLeft n (left, right)
    | abs ((left + n) - right) < 4 = Just (left + n, right)
    | otherwise = Nothing

landRight n (left, right)
    | abs ((right + n) - left) < 4 = Just (left, right + n)
    | otherwise = Nothing

In [51]:
landLeft 2 (0, 0)
landLeft 10 (0, 3)
-- 연산마다 에러 체크

Just (2,0)

Nothing

In [58]:
-- 컨텍스트 없이 값을 받은후, 컨텍스트 값을 반환하는 함수를 활용하기 위해
-- >>= 바인드 사용
(0, 0) |> landRight 1 >>= landLeft 2
-- (>>=) :: m a -> (a -> m b) -> m b  이므로
-- >>= 의 왼편이 m a == (0, 0) |> landRight 1
-- >>= 의 오른편이 (a -> mb) landLeft 2

Nothing >>= landLeft 2

Line 8: Evaluate
Found:
Nothing >>= landLeft 2
Why not:
Nothing

Just (2,1)

Nothing

In [59]:
return (0, 0) >>= landRight 2 >>= landLeft 2 >>= landRight 2

Line 1: Monad law, left identity
Found:
return (0, 0) >>= landRight 2
Why not:
landRight 2 (0, 0)

Just (2,4)

In [61]:
return (0, 0) >>= landLeft 1 >>= landRight 4 >>= landLeft (-1) >>= landRight (-2)
-- 3번째 land 시점에서 Nothing이 되고
-- 4번째 land에서는 Nothing에 적용하므로 Nothing이 된다.

Line 1: Monad law, left identity
Found:
return (0, 0) >>= landLeft 1
Why not:
landLeft 1 (0, 0)

Nothing

In [62]:
banana :: Pole -> Maybe Pole
banana _ = Nothing

In [63]:
return (0, 0) >>= landLeft 1 >>= banana >>= landRight 1

Line 1: Monad law, left identity
Found:
return (0, 0) >>= landLeft 1
Why not:
landLeft 1 (0, 0)

Nothing

In [ ]:
(>>) :: m a -> m b -> m b  
x >> y = x >>= \_ -> y  
-- 디폴트 구현체가 있다
-- 인스턴스에서 구현할 필요가 거의 없는 함수

-- 컨텍스트 값 두개를 받아서 컨텍스트 값을 만드는 함수

-- 앞 컨텍스트값을 m a로 뒤 컨텍스트 값을 함수 반환값으로 변환시켜
-- 바인드>>= 시키는 함수

In [65]:
Nothing >> Just 3 -- 앞 컨텍스트 값이 Nothing
Just 3 >> Just 4
Just 3 >> Nothing

Nothing

Just 4

Nothing

In [66]:
return (0,0) >>= landLeft 1 >> Nothing >>= landRight 1

Line 1: Monad law, left identity
Found:
return (0, 0) >>= landLeft 1
Why not:
landLeft 1 (0, 0)

Nothing

## do 표기법

모나드만을 위한 하스켈의 특별한 구문

In [68]:
Just 3 >>= (\x -> Just (show x ++ "!"))

Just "3!"

In [71]:
Just 3 >>= (\x -> Just "!" >>= (\y -> Just (show x ++ y)))
let x = 3; y = "!" in show x ++ y

Just "3!"

"3!"

In [78]:
Nothing >>= (\x -> Just "!" >>= (\y -> Just (show x ++ y)))
Just 3 >>= (\x -> Nothing >>= (\y -> Just (show x ++ y)))
-- x, y 같은 값이 하나라도 실패 컨텍스트가 되면 실패 컨텍스트로 처리됨
Just 3 >>= (\x -> Just "!" >>= (\y -> Nothing))
-- 실패 컨텍스트가 반환값이라 실패 컨텍스트가 됨

Just 3 >>= (\x -> Nothing >>= (\y -> Just 3))

Line 1: Evaluate
Found:
Nothing >>= (\ x -> Just "!" >>= (\ y -> Just (show x ++ y)))
Why not:
NothingLine 2: Evaluate
Found:
Nothing >>= (\ y -> Just (show x ++ y))
Why not:
NothingLine 4: Use const
Found:
\ y -> Nothing
Why not:
const NothingLine 7: Evaluate
Found:
Nothing >>= (\ y -> Just 3)
Why not:
Nothing

Nothing

Nothing

Nothing

Nothing

In [76]:
foo :: Maybe String
foo = Just 3 >>= (\x ->
      Just "!" >>= (\y ->
      Just (show x ++ y)))
foo

Just "3!"

In [80]:
foo :: Maybe String
foo = do
    x <- Just 3
    y <- Just "!"
    Just (show x ++ y)
foo

Just "3!"

In [84]:
boo = do
    x <- Just 3
    y <- Just "!"
    z <- Nothing
    Just (show x ++ y)
boo
-- 계속 안쪽 람다로 이어붙인것과 같다
-- 연산에 상관없는 z가 Nothing이지만, 반환값은 Nothing이 됨
-- 즉, do 표현식은 모나드 값들을 연결하는 다른 구문일 뿐이다.

Nothing

In [92]:
-- do 표현식에서 let 줄이 아닌 모든 줄은 모나드 값
-- 결과를 알아보기 위해 <- 를 사용
-- do 표현식의 마지막 모나드 값은 결과값에 <-가 사용될 수 없다
-- >>= 연결 코드로의 변환이 되지 않기 때문

Just 9 >>= (\x -> Just (x > 8))

do
    x <- Just 9
    Just (x > 8)

Just True

Just True

In [94]:
routine :: Maybe Pole
routine = do
    start <- return (0, 0)
    first <- landLeft 2 start
    second <- landRight 2 first
    landLeft 1 second
routine

-- do 표현식을 사용하면, 각각의 코드줄은 모나드 값이어야한다.
-- 명령형 코드처럼 보이지만, 각각의 줄에 있는 각각의 값들은
-- 각각의 컨텍스트를 가진 이전 값들의 결과에 의존하는 순차적인 나열일뿐이다.

Line 3: Use let
Found:
start <- return (0, 0)
Why not:
let start = (0, 0)

Just (3,2)

In [96]:
-- 모나드의 특징을 사용하지 않았을 때의 결과
routine :: Maybe Pole  
routine =   
    case Just (0,0) of   
        Nothing -> Nothing  
        Just start -> case landLeft 2 start of  
            Nothing -> Nothing  
            Just first -> case landRight 2 first of  
                Nothing -> Nothing  
                Just second -> landLeft 1 second  

In [101]:
routine :: Maybe Pole  
routine = do  
    start <- return (0,0)  
    first <- landLeft 2 start  
    Nothing  
    -- 모나드 값을 <-에 바인딩하지 않고 작성
    -- 무시하고자하는 결과를 가진 모나드 값 다음에 >>를 두는 것과 같음
    -- 즉슨, Nothing >> second <- land.... 와 같은 표현식이되어
    -- Nothing 결과 값이 된것임
    second <- landRight 2 first  
    landLeft 1 second  
routine

routine2 :: Maybe Pole  
routine2 = do  
    start <- return (0,0)  
    first <- landLeft 2 start  
    return (100,100)
    -- 성공 컨텍스트라서 그냥 무시됨
    second <- landRight 2 first  
    landLeft 1 second  
routine2

Line 3: Use let
Found:
start <- return (0, 0)
Why not:
let start = (0, 0)Line 16: Use let
Found:
start <- return (0, 0)
Why not:
let start = (0, 0)Line 18: Redundant return
Found:
return (100, 100)
Why not:

Nothing

Just (3,2)

## do 표현식의 패턴 매칭과 실패

In [102]:
do
    (x:xs) <- Just "hello"
    return x

Just 'h'

In [103]:
-- 패턴매칭에 실패하면, 다음 패턴이 매칭된다.
-- 모든 패턴 매칭이 실패하면, 에러가 발생하면, 프로그램은 충돌
-- 하지만, let 표현식의 패턴 매칭이 실패하면, 그 즉시 에러 발생
-- 패턴 매칭이 실패할 경우 다음 패턴으로 넘어가는 메커니즘은 let 표현식에 존재하지 않기 때문

In [104]:
-- do 표현식의 패턴 매칭이 실패할 경우 프로그램의 충돌 없이 
-- Manad 타입 클래스에 속해있는 fail 함수가 현재 모나드 컨텍스트에
-- 실패에 대한 결과를 갖게할 수 있다

fail :: (Monad m) => String -> m a  
fail msg = error msg  
-- fail에 대한 디폴트 구현체
-- 기본적으로 충돌한다

In [105]:
:info Maybe
fail _ = Nothing 
-- Maybe 모나드의 fail 구현체는 Nothing 이다.

type Maybe :: * -> *
data Maybe a = Nothing | Just a
  	-- Defined in ‘GHC.Maybe’
instance Functor Maybe -- Defined in ‘GHC.Base’
instance Show a => Show (Maybe a) -- Defined in ‘GHC.Show’
instance Eq a => Eq (Maybe a) -- Defined in ‘GHC.Maybe’
instance Semigroup a => Monoid (Maybe a) -- Defined in ‘GHC.Base’
instance Ord a => Ord (Maybe a) -- Defined in ‘GHC.Maybe’
instance Semigroup a => Semigroup (Maybe a) -- Defined in ‘GHC.Base’
instance Read a => Read (Maybe a) -- Defined in ‘GHC.Read’
instance Applicative Maybe -- Defined in ‘GHC.Base’
instance Foldable Maybe -- Defined in ‘Data.Foldable’
instance Monad Maybe -- Defined in ‘GHC.Base’
instance MonadFail Maybe -- Defined in ‘Control.Monad.Fail’
instance Traversable Maybe -- Defined in ‘Data.Traversable’

In [109]:
do 
    (x:xs) <- Just ("")
    (y:ys) <- Just ("asd")
    return y
-- 패턴 매칭이 실패하면, 전체 컨텍스트가 실패한듯한 효과를 낸다
-- do 표현식이 실패하는 것

Line 2: Redundant bracket
Found:
("")
Why not:
""Line 3: Redundant bracket
Found:
("asd")
Why not:
"asd"

Nothing

## 리스트 모나드

In [113]:
(*) <$> [1,2,3] <*> [10,100,1000]  
-- 리스트는 비결정형 값
-- 인풋, 아웃풋이 모두 비결정형 값

[10,100,1000,20,200,2000,30,300,3000]

In [ ]:
-- 리스트에 대한 모나드 선언
instance Monad [] where  
    return x = [x]  
    -- pure와 같은 작업
    xs >>= f = concat (map f xs)  
    -- 컨텍스트 값과 일반값 처리 함수 받음
    -- concat 함수를 통해 map 결과물을 flatten 함으로 이해
    fail _ = []  
    -- 실패 컨텍스트는 에러가 아닌 []이다

In [114]:
:t concat

concat :: forall (t :: * -> *) a. Foldable t => t [a] -> [a]

In [118]:
[3,4,5] >>= \x -> [x, -x]
-- Maybe의 컨텍스트는 성공 or 실패 였으나
-- 리스트의 컨텍스트는 비결정성이다.
-- 이때 x는 컨텍스트 안의 비결정형 값 중 하나

[3,-3,4,-4,5,-5]

In [120]:
--비결정형 역시 실패 컨텍스트를 가지고 있다
-- []는 Nothing과 매우 유사하다. 결과가 없음을 의미하기 때문
[] >>= \x -> ["bad", "mad", "rad"]
[1,2,3] >>= \x -> []

Line 3: Use const
Found:
\ x -> ["bad", "mad", "rad"]
Why not:
const ["bad", "mad", "rad"]Line 4: Use const
Found:
\ x -> []
Why not:
const []

[]

[]

In [129]:
[1,2] >>= \n -> ['a', 'b'] >>= \ch -> return (n, ch)

do 
    n <- [1,2]
    ch <- ['a', 'b']
    return (n, ch)
    
[(n, ch) | n <- [1,2], ch <- ['a', 'b']]

[(1,'a'),(1,'b'),(2,'a'),(2,'b')]

[(1,'a'),(1,'b'),(2,'a'),(2,'b')]

[(1,'a'),(1,'b'),(2,'a'),(2,'b')]

In [130]:
-- 해설편
-- xs >>= f = concat (map f xs)

-- 1에 대해서
-- 1, "ab" => [(1, 'a'), (1, 'b')]
-- 2에 대해서
-- 2, "ab" => [(2, 'a'), (2, 'b')]

## MonadPlus & guard 함수

In [135]:
[x | x <- [1..50], '7' `elem` show x]

[7,17,27,37,47]

In [134]:
import qualified Control.Monad as Monad
:info Monad.MonadPlus

class (GHC.Base.Alternative m, Monad m) => Monad.MonadPlus m where
  Monad.mzero :: m a
  -- Monoid의 mempty와 비슷 (항등원)
  -- 즉슨, 다형성 상수라는 이야기
  Monad.mplus :: m a -> m a -> m a
  -- Monoid의 mappend와 비슷
  -- 동일한 타입 두개를 받아 또 다른 값을 반환
  -- 어떤 방법으로 두 가지를 이용하여 연산하는 것을 의미

type Monad.MonadPlus :: (* -> *) -> Constraint
class (GHC.Base.Alternative m, Monad m) => Monad.MonadPlus m where
  Monad.mzero :: m a
  Monad.mplus :: m a -> m a -> m a
  	-- Defined in ‘GHC.Base’
instance Monad.MonadPlus [] -- Defined in ‘GHC.Base’
instance Monad.MonadPlus Maybe -- Defined in ‘GHC.Base’
instance Monad.MonadPlus IO -- Defined in ‘GHC.Base’
instance [safe] Control.Monad.Trans.Error.Error e => Monad.MonadPlus (Either e) -- Defined in ‘Control.Monad.Trans.Error’

In [136]:
-- 리스트는 모나드일뿐만 아니라 모노이드이기도 하기 때문에 
-- MonadPlus 타입 클래스의 인스턴스를 만들 수 있다.

:info []
-- instance Monad.MonadPlus [] -- Defined in ‘GHC.Base’
instance MonadPlus [] where  
    mzero = []  
    -- 항등원
    -- 계산이 실패한 비결정형 계산
    mplus = (++)  
    -- mappend
    -- 비결정형 값을 합침

type [] :: * -> *
data [] a = [] | a : [a]
  	-- Defined in ‘GHC.Types’
instance IHaskellDisplay a => IHaskellDisplay [a] -- Defined in ‘IHaskell.Types’
instance Functor [] -- Defined in ‘GHC.Base’
instance Show a => Show [a] -- Defined in ‘GHC.Show’
instance Eq a => Eq [a] -- Defined in ‘GHC.Classes’
instance Monoid [a] -- Defined in ‘GHC.Base’
instance Ord a => Ord [a] -- Defined in ‘GHC.Classes’
instance Semigroup [a] -- Defined in ‘GHC.Base’
instance Read a => Read [a] -- Defined in ‘GHC.Read’
instance Applicative [] -- Defined in ‘GHC.Base’
instance Foldable [] -- Defined in ‘Data.Foldable’
instance Monad [] -- Defined in ‘GHC.Base’
instance MonadFail [] -- Defined in ‘Control.Monad.Fail’
instance Monad.MonadPlus [] -- Defined in ‘GHC.Base’
instance Traversable [] -- Defined in ‘Data.Traversable’

In [141]:
import qualified Control.Monad as Monad

:t Monad.guard

guard :: (MonadPlus m) => Bool -> m ()  

guard True = return ()  
-- True 일 경우 비실행 컨텍스트 내부에 ()
guard False = mzero  
-- False 일 경우 실패 컨텍스트

Monad.guard :: forall (f :: * -> *). Alternative f => Bool -> f ()

In [142]:
import qualified Control.Monad as Monad

Monad.guard (5 > 2) :: Maybe () -- true 
Monad.guard (1 > 2) :: Maybe () -- false
Monad.guard (5 > 2) :: [()] -- true
Monad.guard (1 > 2) :: [()] -- false
-- 어떤 곳에 사용하는 것일까?

Just ()

Nothing

[()]

[]

In [147]:
import qualified Control.Monad as Monad

-- 리스트 모나드에서 비결정형 계산을 필터링하기 위해 이것을 사용한다.
[1..50] >>= (\x -> Monad.guard ('7' `elem` show x) >> return x)
-- >> 왼편은 실패 혹은 성공 컨텍스트를 반환하고
-- >> 오른편은 >> 함수에 의해 실패 혹은 x에 성공 컨텍스트를 씌운 값을 반환한다.
-- 내부의 값보다는(물론 필요하긴 하지만) 성공, 실패 컨텍스트가 중요
-- >> 는 좌항이 성공컨텍스트면 우항의 값을, 실패컨텍스트면 실패를 반환한다

-- 결국 7이 들어가는 element만 map 으로 걸른다음 return 하여 컨텍스트를 씌운후
-- [[],[],[], ..., [7], ... ] 이런 형태
-- concat한 것 (flatten)

[7,17,27,37,47]

In [153]:
import qualified Control.Monad as Monad

-- 아래와 동등하다
-- do notation version
do 
    x <- [1..50]
    Monad.guard ('7' `elem` show x)
    -- 이부분이 >> 연산자로 연결되는 부분이다
    return x
    
-- list comprehension version
[x | x <- [1..50], '7' `elem` show x]
-- 따라서, list comprehension에서의 필터링은 guard를 이용하는 것과 동일하다

[7,17,27,37,47]

[7,17,27,37,47]

In [150]:
import qualified Control.Monad as Monad

Monad.guard (5 > 2) >> return "cool" :: [String]
Monad.guard (1 > 2) >> return "cool" :: [String]

-- guard 자체만으로는 반환 컨텍스트 내부에 들어갈 값을 유지할수는 없다.
-- guard 밖에서 Scope를 유지해야할 것

["cool"]

[]

## 기사의 의무

In [177]:
type KnightPos = (Int, Int)

In [176]:
import qualified Control.Monad as Monad

moveKnight :: KnightPos -> [KnightPos]
moveKnight (c,r) = do  
    (c',r') <- [(c+2,r-1),(c+2,r+1),(c-2,r-1),(c-2,r+1)  
               ,(c+1,r-2),(c+1,r+2),(c-1,r-2),(c-1,r+2)  
               ]  
    -- knight가 이동할 수 있는 모든 방위
    Monad.guard (c' `elem` [1..8] && r' `elem` [1..8])  
    -- 보드 위에 있는지 필터링
    return (c',r')
    -- 반환

In [158]:
--굳이 Monad를 사용하지 않아도 된다.
--하지만 이게 Monad를 사용하지 않는것일까?
moveKnight :: KnightPos -> [KnightPos]  
moveKnight (c,r) = filter onBoard  
    [(c+2,r-1),(c+2,r+1),(c-2,r-1),(c-2,r+1)  
    ,(c+1,r-2),(c+1,r+2),(c-1,r-2),(c-1,r+2)  
    ]  
    where onBoard (c, r) = c `elem` [1..8] && r `elem` [1..8]

In [160]:
moveKnight (6,2)
moveKnight (8,1)

[(8,1),(8,3),(4,1),(4,3),(7,4),(5,4)]

[(6,2),(7,3)]

In [178]:
in3 :: KnightPos -> [KnightPos]
in3 start = do
    first <- moveKnight start
    second <- moveKnight first
    moveKnight second

In [207]:
-- bind 표기법으로는 아래와 같다
-- 이 상황에는 이번 표기법이 제일 좋은것 같다
in3' start = return start >>= moveKnight >>= moveKnight >>= moveKnight

Line 3: Monad law, left identity
Found:
return start >>= moveKnight
Why not:
moveKnight start

In [208]:
in3'' :: KnightPos -> [KnightPos]
in3'' start = [third | 
    first <- moveKnight start,
    second <- moveKnight first,
    third <- moveKnight second]

In [209]:
in3 (3,1)
in3' (3,1)
in3'' (3,1)

[(5,2),(8,3),(6,3),(5,2),(5,4),(8,1),(8,5),(6,1),(6,5),(5,2),(1,2),(4,3),(2,3),(5,2),(5,4),(1,2),(1,4),(4,1),(4,5),(2,1),(2,5),(8,3),(8,5),(4,3),(4,5),(7,2),(7,6),(5,2),(5,6),(6,3),(6,5),(2,3),(2,5),(5,2),(5,6),(3,2),(3,6),(5,2),(1,2),(4,3),(2,3),(5,2),(5,4),(1,2),(1,4),(4,1),(4,5),(2,1),(2,5),(4,3),(4,5),(3,2),(3,6),(1,2),(1,6),(8,1),(8,3),(4,1),(4,3),(7,4),(5,4),(8,3),(8,5),(4,3),(4,5),(7,2),(7,6),(5,2),(5,6),(4,1),(4,3),(3,4),(1,4),(4,3),(4,5),(3,2),(3,6),(1,2),(1,6),(7,2),(3,2),(6,3),(4,3),(7,4),(7,6),(3,4),(3,6),(6,3),(6,7),(4,3),(4,7),(5,2),(1,2),(4,3),(2,3),(5,4),(5,6),(1,4),(1,6),(4,3),(4,7),(2,3),(2,7),(6,1),(6,3),(2,1),(2,3),(5,4),(3,4),(6,3),(6,5),(2,3),(2,5),(5,2),(5,6),(3,2),(3,6),(5,2),(1,2),(4,3),(2,3),(5,4),(5,6),(1,4),(1,6),(4,3),(4,7),(2,3),(2,7),(3,2),(2,3),(3,4),(3,6),(2,3),(2,7)]

[(5,2),(8,3),(6,3),(5,2),(5,4),(8,1),(8,5),(6,1),(6,5),(5,2),(1,2),(4,3),(2,3),(5,2),(5,4),(1,2),(1,4),(4,1),(4,5),(2,1),(2,5),(8,3),(8,5),(4,3),(4,5),(7,2),(7,6),(5,2),(5,6),(6,3),(6,5),(2,3),(2,5),(5,2),(5,6),(3,2),(3,6),(5,2),(1,2),(4,3),(2,3),(5,2),(5,4),(1,2),(1,4),(4,1),(4,5),(2,1),(2,5),(4,3),(4,5),(3,2),(3,6),(1,2),(1,6),(8,1),(8,3),(4,1),(4,3),(7,4),(5,4),(8,3),(8,5),(4,3),(4,5),(7,2),(7,6),(5,2),(5,6),(4,1),(4,3),(3,4),(1,4),(4,3),(4,5),(3,2),(3,6),(1,2),(1,6),(7,2),(3,2),(6,3),(4,3),(7,4),(7,6),(3,4),(3,6),(6,3),(6,7),(4,3),(4,7),(5,2),(1,2),(4,3),(2,3),(5,4),(5,6),(1,4),(1,6),(4,3),(4,7),(2,3),(2,7),(6,1),(6,3),(2,1),(2,3),(5,4),(3,4),(6,3),(6,5),(2,3),(2,5),(5,2),(5,6),(3,2),(3,6),(5,2),(1,2),(4,3),(2,3),(5,4),(5,6),(1,4),(1,6),(4,3),(4,7),(2,3),(2,7),(3,2),(2,3),(3,4),(3,6),(2,3),(2,7)]

[(5,2),(8,3),(6,3),(5,2),(5,4),(8,1),(8,5),(6,1),(6,5),(5,2),(1,2),(4,3),(2,3),(5,2),(5,4),(1,2),(1,4),(4,1),(4,5),(2,1),(2,5),(8,3),(8,5),(4,3),(4,5),(7,2),(7,6),(5,2),(5,6),(6,3),(6,5),(2,3),(2,5),(5,2),(5,6),(3,2),(3,6),(5,2),(1,2),(4,3),(2,3),(5,2),(5,4),(1,2),(1,4),(4,1),(4,5),(2,1),(2,5),(4,3),(4,5),(3,2),(3,6),(1,2),(1,6),(8,1),(8,3),(4,1),(4,3),(7,4),(5,4),(8,3),(8,5),(4,3),(4,5),(7,2),(7,6),(5,2),(5,6),(4,1),(4,3),(3,4),(1,4),(4,3),(4,5),(3,2),(3,6),(1,2),(1,6),(7,2),(3,2),(6,3),(4,3),(7,4),(7,6),(3,4),(3,6),(6,3),(6,7),(4,3),(4,7),(5,2),(1,2),(4,3),(2,3),(5,4),(5,6),(1,4),(1,6),(4,3),(4,7),(2,3),(2,7),(6,1),(6,3),(2,1),(2,3),(5,4),(3,4),(6,3),(6,5),(2,3),(2,5),(5,2),(5,6),(3,2),(3,6),(5,2),(1,2),(4,3),(2,3),(5,4),(5,6),(1,4),(1,6),(4,3),(4,7),(2,3),(2,7),(3,2),(2,3),(3,4),(3,6),(2,3),(2,7)]

In [210]:
-- 처음과 끝을 받아 세번만에 갈수 있는가를 판단하는 함수
canReachIn3 :: KnightPos -> KnightPos -> Bool
canReachIn3 start end = end `elem` in3 start

In [213]:
(6,2) `canReachIn3` (6,1)
(6,2) `canReachIn3` (7,3)

True

False

## 모나드 규칙

### 1. 좌항등원

`return x >>= f` = `f x`

In [4]:
-- Maybe의 최소 컨텍스트(return)이 Just 이므로 이해하기 쉽다

return 3 >>= (\x -> Just (x+100000))
(\x -> Just (x+100000)) 3

Line 3: Monad law, left identity
Found:
return 3 >>= (\ x -> Just (x + 100000))
Why not:
(\ x -> Just (x + 100000)) 3

Just 100003

Just 100003

In [9]:
return "WoM" >>= (\x -> [x,x,x])
-- 리스트 위에 리스트 컨텍스트를 다시 씌웠다
concat $ map (\x -> [x,x,x]) $ return "WoM"
(\x -> [x,x,x]) "WoM"

Line 1: Monad law, left identity
Found:
return "WoM" >>= (\ x -> [x, x, x])
Why not:
(\ x -> [x, x, x]) "WoM"Line 3: Use concatMap
Found:
concat $ map (\ x -> [x, x, x]) $ return "WoM"
Why not:
concatMap (\ x -> [x, x, x]) (return "WoM")

["WoM","WoM","WoM"]

["WoM","WoM","WoM"]

["WoM","WoM","WoM"]

### 2. 우항등원

`m >>= return` = `m`

In [4]:
-- m이 >>=의 인자라는 것은, 이미 컨텍스트가 있음을 의미하고 
-- return 됨음, 컨텍스트를 다시 씌워져서 반환됨을 의미한다
-- 즉슨 컨텍스트를 벗겼다가 씌우면 내버려둔것과 같음을 의미

Nothing >>= return -- point free
Just "move on up" >>= (\x -> return x)
[1,2,3,4] >>= (\x -> return x)
concat $ map (\x -> return x) [1,2,3,4]
putStrLn "Wah!" >>= (\x -> return x)

Line 5: Monad law, right identity
Found:
Nothing >>= return
Why not:
NothingLine 5: Evaluate
Found:
Nothing >>= return
Why not:
NothingLine 6: Avoid lambda
Found:
(\ x -> return x)
Why not:
returnLine 7: Avoid lambda
Found:
(\ x -> return x)
Why not:
returnLine 8: Use concatMap
Found:
concat $ map (\ x -> return x) [1, 2, 3, 4]
Why not:
concatMap (\ x -> return x) [1, 2, 3, 4]Line 8: Avoid lambda
Found:
(\ x -> return x)
Why not:
returnLine 9: Avoid lambda
Found:
(\ x -> return x)
Why not:
return

Nothing

Just "move on up"

[1,2,3,4]

[1,2,3,4]

Wah!

### 3. 결합성

`(m >>= f) >>= g` = `m >>= (\x -> f x >>= g)`

-- >>= g 입장에서는 컨텍스트 값을 받아야 하므로 우항과 같은 표현이 됨

-- 또한 m >>= 입장에서는 함수를 받아야하므로 우항과 같은 표현이 됨

In [21]:
return (0, 0) >>= landRight 2 >>= landLeft 2 >>= landRight 2

-- 좌항부터 정리
((return (0,0) >>= landRight 2) >>= landLeft 2) >>= landRight 2
-- Just (0, 2) ...
-- Just (2, 2) ...
-- Just (2, 4)

-- 우항부터 정리
-- 우항부터 하면 단순히 함수 내부가 합쳐지는 형태가 된다.
return (0,0) >>= (\x -> 
    landRight 2 x >>= (\y -> 
    landLeft 2 y >>= (\z -> 
    landRight 2 z)))
-- ... (\y -> landRight 2 ())

Line 1: Monad law, left identity
Found:
return (0, 0) >>= landRight 2
Why not:
landRight 2 (0, 0)Line 4: Monad law, left identity
Found:
return (0, 0) >>= landRight 2
Why not:
landRight 2 (0, 0)Line 11: Monad law, left identity
Found:
return (0, 0)
  >>=
    (\ x
       -> landRight 2 x
            >>= (\ y -> landLeft 2 y >>= (\ z -> landRight 2 z)))
Why not:
(\ x
   -> landRight 2 x
        >>= (\ y -> landLeft 2 y >>= (\ z -> landRight 2 z)))
  (0, 0)Line 11: Use >=>
Found:
\ x
  -> landRight 2 x
       >>= (\ y -> landLeft 2 y >>= (\ z -> landRight 2 z))
Why not:
landRight 2
  Control.Monad.>=> (\ y -> landLeft 2 y >>= (\ z -> landRight 2 z))Line 12: Use >=>
Found:
\ y -> landLeft 2 y >>= (\ z -> landRight 2 z)
Why not:
landLeft 2 Control.Monad.>=> (\ z -> landRight 2 z)Line 13: Avoid lambda
Found:
\ z -> landRight 2 z
Why not:
landRight 2

Just (2,4)

Just (2,4)

Just (2,4)

In [45]:
import Control.Monad

:t (.)

-- 우항결합을 통해 m a >>= (a -> m b) >>= (b -> m c) >>= (c -> m d)의 연속을
-- m a >>= (a -> m d)로 최소화한다
:t (<=<)
-- Right-to-left composition of Kleisli arrows
-- forall (m :: * -> *) b c a. Monad m => (b -> m c) -> (a -> m b) -> a -> m c
-- f <=< g = (\x -> g x >>= f)
-- 두번째 인자 g: 첫번째로 인자를 받을 모나드용 함수
-- 첫번째 인자 f: 두번째로 인자를 받을 모나드용 함수
-- g <=< f 라고 써놓는게 이해가 빠를듯 (우에서 좌로 데이터가 이등)
-- 순서와 용법이 .과 같다

:t (>=>)
-- Left-to-right composition of Kleisli arrows.
-- <=< 의 반대
-- f >=> g 이다
-- 개인적으로 더 마음에 든다

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

(<=<) :: forall (m :: * -> *) b c a. Monad m => (b -> m c) -> (a -> m b) -> a -> m c

(>=>) :: forall (m :: * -> *) a b c. Monad m => (a -> m b) -> (b -> m c) -> a -> m c

In [46]:
f x = [x, -x]
g x = [x*3, x*2]
h = f <=< g
h 3 -- g 적용 후 f 적용

[9,-9,6,-6]

In [51]:
-- f <=< (g <=< h)는 (f <=< g) <=< h 와 같아야함

-- f <=< return 은 f 와 같다
-- (f <=< return) x
-- f (return x)
-- f x 이므로 (또 point-free)
-- f 이다.

In [53]:
-- 이는 f . (g . h)가 (f . g). h 와 같으며
-- f . id 는 id. f,  f 와 같은 것과 비슷하다